In [28]:
### Pyhton 3.7.1 ###
#-----L5 2-way ANOVA------
#~~packages
import pandas as pd
import scipy.stats as stats
import numpy as np
#time
import time
tStart = time.time()
#install scipy
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
#install matplotlib
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison 
#data visualization
import seaborn as sns
#R packages
import rpy2.robjects as robjects
#pingouin (ANOVA)
import pingouin as pi

In [29]:
# python
#~~ loading data
df_raw = pd.read_csv(r"C:\Users\johnson_yang\Desktop\WOW_data_forStudent.csv")#改成自己的檔案位置

#~~ checking data
df = df_raw[['ID','gender']]
print('---------------------Data type---------------------')
print(df.dtypes)
print('---------------------ID info---------------------')
df[['ID']]=df[['ID']].astype('object')
print(df.dtypes)


---------------------Data type---------------------
ID        int64
gender    int64
dtype: object
---------------------ID info---------------------
ID        object
gender     int64
dtype: object


C:\Users\johnson_yang\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [30]:
# R
#~~loading data and checking data
rr=robjects.r
rr('CHI.raw <- read.csv("C:/Users/johnson_yang/Desktop/WOW_data_forStudent.csv",header=TRUE)')
rr('CHI.goodness<-CHI.raw[c(2,4)]')
rr('CHI.goodness$ID <- as.factor(CHI.goodness$ID)')
rr('install.packages("plyr")')
rr('install.packages("dplyr")')
rr('library(plyr)')
rr('library(dplyr)')
rr('CHI.goodness$gender <- mapvalues(CHI.goodness$gender,from = c(0,1),to = c("female", "male"))')
rr('CHI.goodness$gender <-as.factor(CHI.goodness$gender)')

female,female,female,male,...,female,female,male,male


In [31]:
# python chi

#from scipy.stats import chisquare
#from scipy.stats import chi2_contingency
#df_gender_array=np.array(df[['gender']]).reshape(-1)
#print(chisquare(df_gender_array))



#R chi

rr('count<-table(CHI.goodness$gender)')
print(rr('count'))
print('--------------------------------------------------')
print(rr('chisq.test(count)'))




female   male 

    89    104 

--------------------------------------------------


	Chi-squared test for given probabilities



data:  count

X-squared = 1.1658, df = 1, p-value = 0.2803





In [32]:
#R chi with probability

# 50%女生(female)、50%男生(male)
rr('null.probs.1 = c(0.5,0.5)')
print(rr('chisq.test(count, p=null.probs.1)'))
print('--------------------------------------------------')
# 40%女生(female)、60%男生(male)
rr('null.probs.2 = c(0.4,0.6)')
print(rr('chisq.test(count, p=null.probs.2)'))



	Chi-squared test for given probabilities



data:  count

X-squared = 1.1658, df = 1, p-value = 0.2803



--------------------------------------------------


	Chi-squared test for given probabilities



data:  count

X-squared = 3.006, df = 1, p-value = 0.08295





In [33]:
# R Test of independence

rr('CHI.ind<-CHI.raw[c(2,3,4)]')
rr('CHI.ind$ID <- as.factor(CHI.ind$ID)')
rr('CHI.ind$gender <- mapvalues(CHI.ind$gender,from = c(0,1),to = c("female", "male"))')
rr('CHI.ind$gender <-as.factor(CHI.ind$gender)')

# chi of independence
rr('ind.table <- table(CHI.ind$gender,CHI.ind$ethnic)')
print(rr('ind.table'))

        

         Human  Orc  Undead 

  female     38   19      32

  male       49   23      32



In [34]:
# R of 2 by 2 contingency table (相依)

rr('CHI.McNemar<-CHI.raw[c(2,12,22)]')
rr('CHI.McNemar$ID <- as.factor(CHI.McNemar$ID)')
print(rr('summary(CHI.McNemar)'))
print('--------------------------------------------------')
# mutate

rr('CHI.McNemar<-mutate(CHI.McNemar,reading3 = ifelse(reading_3 >= 499.3,"3H","3L"))')
#print('--------------------------------------------------')
rr('CHI.McNemar<-mutate(CHI.McNemar,reading1 = ifelse(reading_1 >= 477.3,"1H","1L"))')
#print('--------------------------------------------------')
# MC.test
rr('MC.test <- table(CHI.McNemar$reading3,CHI.McNemar$reading1)')
print(rr('MC.test'))
print('--------------------------------------------------')
# chi
print(rr('mcnemar.test(MC.test)'))

       ID        reading_3       reading_1    

 1041001:  1   Min.   :423.0   Min.   :393.0  

 1041002:  1   1st Qu.:489.0   1st Qu.:467.0  

 1041003:  1   Median :503.0   Median :481.0  

 1041004:  1   Mean   :499.3   Mean   :477.3  

 1041005:  1   3rd Qu.:512.0   3rd Qu.:489.0  

 1041006:  1   Max.   :543.0   Max.   :530.0  

 (Other):187                                  

--------------------------------------------------
    

     1H 1L

  3H 94 11

  3L 22 66

--------------------------------------------------


	McNemar's Chi-squared test with continuity correction



data:  MC.test

McNemar's chi-squared = 3.0303, df = 1, p-value = 0.08172





In [35]:
# R of Fisher’s exact probability test

rr('TeaTasting <- matrix(c(4, 0, 0, 4),nrow = 2,dimnames = list(Guess = c("Milk_First_Guess", "Tea_First_Guess"),Truth = c("Milk_First_Truth", "Tea_First_Truth")))')
print(rr('TeaTasting'))
print('--------------------------------------------------')
print(rr('fisher.test(TeaTasting)'))
print('--------------------------------------------------')
# Yate's correlation for continuity
rr('TeaTasting.2 <- matrix(c(10, 0, 0, 10),nrow = 2,dimnames = list(Guess = c("Milk_First_Guess", "Tea_First_Guess"),Truth = c("Milk_First_Truth", "Tea_First_Truth")))')
print(rr('TeaTasting.2'))
print('--------------------------------------------------')
print(rr('chisq.test(TeaTasting.2)'))

                  Truth

Guess              Milk_First_Truth Tea_First_Truth

  Milk_First_Guess                4               0

  Tea_First_Guess                 0               4

--------------------------------------------------


	Fisher's Exact Test for Count Data



data:  TeaTasting

p-value = 0.02857

alternative hypothesis: true odds ratio is not equal to 1

95 percent confidence interval:

 1.339059      Inf

sample estimates:

odds ratio 

       Inf 



--------------------------------------------------
                  Truth

Guess              Milk_First_Truth Tea_First_Truth

  Milk_First_Guess               10               0

  Tea_First_Guess                 0              10

--------------------------------------------------


	Pearson's Chi-squared test with Yates' continuity correction



data:  TeaTasting.2

X-squared = 16.2, df = 1, p-value = 5.699e-05



